# Playing around with Enformer and enformer_loader

## We can run VSCode in the farm!

In [2]:
import os

cwd = os.getcwd()
print(cwd)

/lustre/scratch126/gengen/teams/parts/sb79/enformer_loader_sgb


In [3]:
import socket

node_name = socket.gethostname()
print(node_name)

node-13-17


In [7]:
import multiprocess
cores = multiprocess.cpu_count()
print("Cores Available: " + str(cores))

Cores Available: 256


In [ ]:
# Test R code to check connection to a node
print("Hello, R is running successfully!")

# Check the R version
R.version.string

# Check the current working directory
getwd()
# Generate a simple plot
x <- rnorm(100)
y <- rnorm(100)
plot(x, y, main="Scatter plot of random points", xlab="X-axis", ylab="Y-axis")
quartz()  # Opens a new window for the plot (on Linux or macOS)
plot(1:10, 1:10)

R code is not that well suited to VSCode... but should be able to write scripts here anyway :)

## Let's inspect the enformer_loader output

In [16]:
# Import necessary packages
import numpy as np
import sys
import os
import pandas as pd
from Bio import SeqIO
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import math
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import re
import csv

/software/hgi/installs/conda-audited/miniforge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
#sequence import
import pysam
import pyfaidx

# import fasta file and create function for retrieving sequences
fa_loc = '/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_full/fasta/GRCh38.GRCg6a.full.renamed.merged.fa'
pyfaidx.Faidx(fa_loc)
fasta_open = pysam.Fastafile(fa_loc)

In [36]:
chromlength = []
chromname = []
with pysam.Fastafile(fa_loc) as fasta:
    for name in fasta.references:
        chromname.append(name)
        length = fasta.get_reference_length(name)
        chromlength.append(length)
        print(f"Chromosome {name} has length {length}")

Chromosome hg38_1 has length 248956422
Chromosome hg38_10 has length 133797422
Chromosome hg38_11 has length 135086622
Chromosome hg38_12 has length 133275309
Chromosome hg38_13 has length 114364328
Chromosome hg38_14 has length 107043718
Chromosome hg38_15 has length 101991189
Chromosome hg38_16 has length 90338345
Chromosome hg38_17 has length 83257441
Chromosome hg38_18 has length 80373285
Chromosome hg38_19 has length 58617616
Chromosome hg38_2 has length 242193529
Chromosome hg38_20 has length 64444167
Chromosome hg38_21 has length 46709983
Chromosome hg38_22 has length 50818468
Chromosome hg38_3 has length 198295559
Chromosome hg38_4 has length 190214555
Chromosome hg38_5 has length 181538259
Chromosome hg38_6 has length 170805979
Chromosome hg38_7 has length 159345973
Chromosome hg38_8 has length 145138636
Chromosome hg38_9 has length 138394717
Chromosome hg38_MT has length 16569
Chromosome hg38_X has length 156040895
Chromosome hg38_Y has length 57227415
Chromosome hg38_KI27072

In [37]:
# write to chromosome lengths file
output_file = "all_chrom_len.txt"
# Open the file in write mode
with open(output_file, "w") as file:
    # Zip the two vectors together and write them to the file
    for n, l in zip(chromname, chromlength):
        file.write(f"{n}\t{l}\n")



In [38]:
# write to chromosome lengths file
output_file = "all_chrom_names.txt"
# Open the file in write mode
with open(output_file, "w") as file:
    # Zip the two vectors together and write them to the file
    for n, l in zip(chromname, chromlength):
        file.write(f"{n}\n")



In [18]:
# let's import the .bed file from enformer_loader
path = 'tests/data/'
bed_df = pd.read_csv(path + 'test_dataset.bed', sep='\t', header=None)
bed_df

,0,1,2,3
0,chr1,880,920,"[1.0, 1.0]"
1,chr1,493,533,"[1.0, 1.0]"
2,chr2,390,430,"[2.200000047683716, 3.200000047683716]"
3,chr1,92,132,"[1.0, 1.0]"
4,chr1,458,498,"[1.0, 1.0]"
...,...,...,...,...
95,chr3,563,603,"[2.0, 2.0]"
96,chr1,319,359,"[1.0, 1.0]"
97,chr2,477,517,"[3.200000047683716, 3.200000047683716]"
98,chr2,21,61,"[0.800000011920929, 0.800000011920929]"


In [21]:
# see how we could generate a train/val/test dataset for enformer
gtf_file = "/lustre/scratch126/gengen/projects/graft/Dataset/reference/hg38_galGal6_chr1/gtf/GRCh38.GRCg6a.chr1.110.gtf"
gtf_df = pd.read_csv(gtf_file, sep="\t", comment="#", header=None)
# Give appropriate column names
gtf_df.columns = ['seqname', 'source', 'feature', 'start', 'end', '.', 'strand', '.', 'attributes']
gtf_df['seqname'].unique()

array(['hg38_1', 'hg38_10', 'hg38_11', 'hg38_12', 'hg38_13', 'hg38_14',
       'hg38_15', 'hg38_16', 'hg38_17', 'hg38_18', 'hg38_19', 'hg38_2',
       'hg38_20', 'hg38_21', 'hg38_22', 'hg38_3', 'hg38_4', 'hg38_5',
       'hg38_6', 'hg38_7', 'hg38_8', 'hg38_9', 'hg38_GL000009.2',
       'hg38_GL000194.1', 'hg38_GL000195.1', 'hg38_GL000205.2',
       'hg38_GL000213.1', 'hg38_GL000216.2', 'hg38_GL000218.1',
       'hg38_GL000219.1', 'hg38_GL000220.1', 'hg38_GL000225.1',
       'hg38_KI270442.1', 'hg38_KI270711.1', 'hg38_KI270713.1',
       'hg38_KI270721.1', 'hg38_KI270726.1', 'hg38_KI270727.1',
       'hg38_KI270728.1', 'hg38_KI270731.1', 'hg38_KI270733.1',
       'hg38_KI270734.1', 'hg38_KI270744.1', 'hg38_KI270750.1', 'hg38_MT',
       'hg38_X', 'hg38_Y', 'gg6_1'], dtype=object)

In [22]:
gtf_df.iloc[0:10,:]

,seqname,source,feature,start,end,.,strand,.,attributes
0,hg38_1,havana,gene,11869,14409,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; g..."
1,hg38_1,havana,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
2,hg38_1,havana,exon,11869,12227,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
3,hg38_1,havana,exon,12010,12057,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
4,hg38_1,havana,transcript,12010,13670,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
5,hg38_1,havana,gene,12010,13670,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; g..."
6,hg38_1,havana,exon,12179,12227,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
7,hg38_1,havana,exon,12613,12721,.,+,.,"gene_id ""ENSG00000290825""; gene_version ""1""; t..."
8,hg38_1,havana,exon,12613,12697,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."
9,hg38_1,havana,exon,12975,13052,.,+,.,"gene_id ""ENSG00000223972""; gene_version ""6""; t..."


In [25]:
gtf_df.shape

(3629944, 9)

# Now let's try to create dataset for H3K4me3, with hg38_1 as train and hg38_2 as validation

We have test_1_valchr.txt containing the name of the validation chromosome (hg38_2)


In [ ]:
# we can o